In [17]:
import pandas as pd
import numpy as np
import re
import requests
from io import StringIO
import sqlalchemy as alch
from getpass import getpass
from sqlalchemy import create_engine


In [4]:
columns_to_import = ['idno','cntry','imsmetn','imdfetn','impcntr','imbgeco', 'imueclt', 'imwbcnt','anctry1','anctry2']

df = pd.read_csv("../data/ESS10.csv", usecols=columns_to_import)
df

,idno,cntry,imsmetn,imdfetn,impcntr,imbgeco,imueclt,imwbcnt,anctry1,anctry2
0,10002,BG,3,3,3,2,2,2,14030,555555
1,10006,BG,1,2,2,7,5,4,14030,555555
2,10009,BG,1,3,2,6,7,6,14030,555555
3,10024,BG,1,3,2,88,88,5,14030,555555
4,10027,BG,1,3,4,4,5,3,14030,555555
...,...,...,...,...,...,...,...,...,...,...
33346,27808,SK,2,2,3,0,8,6,15130,888888
33347,27826,SK,2,2,2,7,7,6,15130,555555
33348,27834,SK,4,4,3,0,2,2,15130,555555
33349,27846,SK,4,9,4,2,8,1,15130,555555


In [5]:
names = df.columns
names

Index(['idno', 'cntry', 'imsmetn', 'imdfetn', 'impcntr', 'imbgeco', 'imueclt',
       'imwbcnt', 'anctry1', 'anctry2'],
      dtype='object')

In [6]:
#reemplacamos los valores 77,88,99 (no sabe, no contesta) por valores nan
df['imbgeco'] = df['imbgeco'].replace([77, 88, 99], np.nan)
df['imueclt'] = df['imueclt'].replace([77, 88, 99], np.nan)
df['imwbcnt'] = df['imwbcnt'].replace([77, 88, 99], np.nan)
df['impcntr'] = df['impcntr'].replace([7, 8, 9], np.nan)
df['imsmetn'] = df['imsmetn'].replace([7, 8, 9], np.nan)
df['imdfetn'] = df['imdfetn'].replace([7, 8, 9], np.nan)

In [7]:
df.drop_duplicates(inplace=True)

In [8]:
ethnicities = """10000	European
11000	West European
11010	Austrian nfs
11018	Austrian city or region nec
11019	Austrian nec
11020	Belgian nfs
11021	Flemish
11022	Walloon
11023	Brussels/Bruxellois
11028	Belgian city or region nec
11029	Belgian nec
11030	British nfs
11031	English
11032	Northern Irish
11033	Scottish
11034	Ulster
11035	Welsh
11038	British city or region nec
11039	British nec
11040	Dutch nfs
11048	Dutch city or region nec
11049	Dutch nec
11050	French nfs
11051	Breton
11052	Corse
11058	French city or region nec
11059	French nec
11060	Frisian
11070	German nfs
11077	German diaspora/ethnic Germans
11078	German city or region nec
11079	German nec
11080	Irish nfs
11081	Irish traveller
11088	Irish city or region nec
11089	Irish nec
11090	Luxembourgish nfs
11098	Luxembourgish city or region nec
11099	Luxembourgish nec
11100	Swiss nfs
11101	Swiss language region nec
11102	Swiss Canton
11108	Swiss city or region nec
11109	Swiss nec
11110	Yeniche
11990	West European nec
12000	North European (Nordic)
12010	Danish nfs
12011	Faroese
12012	Greenlander
12018	Danish city or region nec
12019	Danish nec
12020	Finnish nfs
12021	Swedish-speaking Finnish
12022	Ingrian
12028	Finnish city or region nec
12029	Finnish nec
12030	Icelandic nfs
12038	Icelandic city or region nec
12039	Icelandic nec
12040	Norwegian nfs
12048	Norwegian city or region nec
12049	Norwegian nec
12050	Sami
12060	Swedish nfs
12068	Swedish city or region nec
12069	Swedish nec
12990	North European nec
13000	South European
13010	Basque
13020	Greek Cypriot nfs
13028	Greek Cypriot city or region nec
13029	Greek Cypriot nec
13030	Greek nfs
13031	Greek Muslim
13038	Greek city or region nec
13039	Greek nec
13040	Italian nfs
13041	Ladin
13042	Sicilian
13043	South Tyrolian
13047	Italian diaspora/ethnic Italian
13048	Italian city or region nec
13049	Italian nec
13050	Maltese
13060	Portuguese nfs
13068	Portuguese city or region nec
13069	Portuguese nec
13070	Spanish nfs
13071	Andalusian
13072	Balearic
13073	Canarian
13074	Catalan
13075	Galician
13076	Navarran
13077	Valencian
13078	Spanish city or region nec
13079	Spanish nec
13990	South European nec
14000	South-East European
14010	Albanian nfs
14018	Albanian city or region nec
14019	Albanian nec
14020	Bosniak/Bosnian
14030	Bulgarian nfs
14031	Bulgarian Muslim
14038	Bulgarian city or region nec
14039	Bulgarian nec
14040	Croatian nfs
14048	Croatian city or region nec
14049	Croatian nec
14050	Kosovar nfs
14058	Kosovar city or region nec
14059	Kosovar nec
14060	Macedonian nfs
14061	Macedonian Muslim
14068	Macedonian city or region nec
14069	Macedonian nec
14070	Moldovan
14080	Montenegrin nfs
14081	Montenegrin Muslim
14088	Montenegrin city or region nec
14089	Montenegrin nec
14090	Romanian nfs
14098	Romanian city or region nec
14099	Romanian nec
14100	Roma/Gypsy/Sinti
14110	Serbian nfs
14111	Bosnian Serb
14117	Non-migrant Serbs living outside of Serbia
14119	Serbian nec
14120	Slovene nfs
14128	Slovenian city or region nec
14129	Slovene nec
14130	Vlach
14140	Ashkali/Balkan Egyptian
14150	Gorani
14160	Yugoslav
14170	Bunjevac
14990	South-East European nec
15000	East European
15010	Belarusian
15020	Czech nfs
15028	Czech city or region nec
15029	Czech nec
15030	Estonian nfs
15038	Estonian city or region nec
15039	Estonian nec
15040	Hungarian nfs
15047	Hungarian diaspora/ethnic Hungarian
15048	Hungarian city or region nec
15049	Hungarian nec
15050	Latvian nfs
15051	Latgalian
15058	Latvian city or region nec
15059	Latvian nec
15060	Lithuanian nfs
15068	Lithuanian city or region nec
15069	Lithuanian nec
15080	Moravian
15090	Polish nfs
15091	Lemko
15092	Karaim
15093	Kashubian
15098	Polish city or region nec
15099	Polish nec
15100	Ruthenian/Rusyn
15110	Russian nfs
15111	Estonian Russian
15112	Latvian Russian
15113	Ukrainian Russian
15117	Russian diaspora/ethnic Russian
15118	Russian city or region nec
15119	Russian nec
15120	Silesian
15130	Slovak nfs
15138	Slovak city or region nec
15139	Slovak nec
15140	Ukrainian nfs
15148	Ukrainian city or region nec
15149	Ukrainian nec
15150	Sorbian/Wendish
15990	East European nec
20000	North African, Middle Eastern and Central Asian
21000	Arab
21010	Algerian
21020	Arab Israeli
21030	Bahraini
21040	Bedouin
21050	Egyptian
21060	Emirati
21070	Iraqi
21080	Jordanian
21090	Kuwaiti
21100	Lebanese
21110	Libyan
21120	Mauritanian
21130	Moroccan
21140	Omani
21150	Palestinian
21160	Qatari
21170	Sahrawi/Sahraoui
21180	Saudi Arabian
21190	Syrian
21200	Tunisian
21210	Yemeni
21220	Muslim nfs
21990	Arab nec
22000	Jewish
22010	Jewish Israeli nfs
22011	Ashkenazi
22012	Sephardi
22013	Mizrahi
22990	Jewish nec
23000	Turkish
23001	Turkish Cypriot
23007	Turkish diaspora/ethnic Turk
23008	Turkish city or region nec
23009	Turkish nec
24000	Iranian and Central Asian
24010	Afghan
24020	Azeri/Azerbaijani
24030	Hazara
24040	Iranian
24050	Kazakh
24060	Kyrgyz
24070	Pathan
24080	Tajik
24090	Tatar
24100	Turkmen
24110	Uighur
24120	Uzbek
24130	Bashkir
24140	Chuvash
24150	Mordva
24990	Iranian and Central Asian nec
25000	Other North African and Middle Eastern
25010	Armenian
25020	Assyrian/Chaldean
25030	Berber
25040	Coptic
25050	Druze
25060	Georgian
25070	Israeli nfs
25078	Israeli city or region nec
25079	Israeli nec
25080	Kurdish
25090	Mandaean
25100	Avar
25110	Chechen
25120	Maronite
25990	Other North African and Middle Eastern nec
30000	Sub-Saharan African
31000	West and Central African
31010	Akan
31020	Angolan
31030	Beninese
31040	Bissau-Guinean
31050	Burkinabe
31060	Burundian
31070	Cabo Verdean
31080	Cameroonian
31090	Central African
31100	Chadian
31110	Congolese
31120	Ghanaian
31130	Guinean
31140	Ivorian
31150	Liberian
31160	Malian
31170	Nigerian
31180	Rwandan
31190	Senegalese
31200	Sierra Leonean
31210	Yoruban
31990	West and Central African nec
32000	Africa's Horn
32010	Afar
32020	Amhara
32030	Djiboutian
32040	Eritrean
32050	Ethiopian
32060	Fur
32070	Nubian
32080	Oromo
32090	Somali
32100	Sudanese
32110	Tigre
32120	Tigray-Tigrinya
32990	Africa's Horn nec
33000	East and South African
33010	Comoran
33020	Dinka
33030	Kenyan
33040	Malawian
33050	Masai
33060	Motswana
33070	Mozambican
33080	Nuer
33090	Seychellois
33100	South African nfs
33101	Afrikaaner
33102	British South African
33109	South African nec
33110	Tanzanian
33120	Ugandan
33130	Zambian
33140	Zimbabwean
33150	Zulu
33160	Malagasy
33990	East and South African nec
40000	South and South-East Asian
41000	South Asian
41010	Bangladeshi
41020	Bhutanese
41030	Indian nfs
41031	Bengali
41032	Gujerati
41033	Malayali
41034	Punjabi
41035	Sikh
41036	Telegu
41037	Indian diaspora
41038	Indian city or region nec
41039	Indian nec
41040	Kashmiri
41050	Maldivian
41060	Mauritian
41070	Nepalese
41080	Pakistani nfs
41081	Sindhi
41088	Pakistani city or region nec
41089	Pakistani nec
41090	Parsi
41100	Rohingya
41110	Sri Lankan nfs
41111	Sinhalese
41112	Burgher
41118	Sri Lankan city or region nec
41119	Sri Lankan nec
41120	Tamil
41990	South Asian nec
42000	Mainland and Buddhist South-East Asian
42010	Balinese
42020	Burmese/Bamar
42030	Hmong
42040	Karen
42050	Khmer
42060	Lao
42070	Mon
42080	Thai
42090	Vietnamese
42990	Mainland and Buddhist South-East Asian nec
43000	Maritime and Muslim South-East Asian
43010	Bruneian
43020	Indonesian nfs
43021	Acehnese
43022	Javanese
43023	Madurese
43024	Moluccan
43025	Sundanese
43028	Indonesian city or region nec
43029	Indonesian nec
43030	Malay nfs
43031	Kadazan
43039	Malay nec
43040	Timorese
43990	Maritime and Muslim South-East Asian nec
50000	East Asian
51000	Chinese Asian
51010	Chinese nfs
51011	Hong Kong Chinese
51012	Chinese diaspora
51019	Chinese nec
51020	Singaporean Chinese
51030	Taiwanese Chinese
51990	Chinese Asian nec
52000	North-East Asian
52010	Japanese nfs
52011	Ainu
52012	Burakamin
52019	Japanese nec
52020	Korean
52030	Mongolian
52040	Tibetan
52990	North-East Asian nec
60000	Latin American
60010	Amerindian (indigenous peoples of South and Central America nec)
61000	South American
61020	Argentinian
61030	Aymara
61040	Bolivian
61050	Brazilian
61060	Chilean
61070	Colombian
61080	Ecuadorian
61090	Paraguayan
61100	Peruvian
61110	Quechuan
61120	Uruguayan
61130	Venezuelan
61990	South American nec
62000	Central American
62010	Costa Rican
62020	Cuban
62030	Dominican (Dominican Republic)
62040	Filipino
62050	Guatemalan
62060	Honduran
62070	Mayan
62080	Mexican
62090	Nicaraguan
62100	Panamanian
62110	Puerto Rican
62120	Salvadorean
62990	Central American nec
70000	Caribbean
71000	English-speaking Caribbean
71010	Anguillan
71020	Antiguan
71030	Bahamian
71040	Barbadian
71050	Barbudan
71060	Dominican (Dominica)
71070	Guyanese
71080	Jamaican
71090	Trinidadian
71990	English-speaking Caribbean nec
72000	French or Dutch-speaking Caribbean
72010	Antillais
72020	Aruban
72030	Curacaoan
72040	Haitian
72050	Surinamese
72990	French or Dutch-speaking Caribbean nec
80000	North American and Australasian
81000	North American
81010	American nfs
81011	African American
81012	Hispanic American
81013	Asian American
81019	American nec
81020	Bermudan
81030	Canadian nfs
81031	Quebecois/French Canadian
81032	Black Canadian
81033	Asian Canadian
81039	Canadian nec
81040	Native North American Indian
81990	North American nec
82000	Australasian
82010	Australian nfs
82011	Australian Aboriginal
82012	Australian South Sea Islander
82013	Torres Strait Islander
82019	Australian nec
82020	Fijian/iTaukei
82030	Maori
82040	New Zealander
82050	Papua New Guinean
82060	Samoan
82070	Tahitian (and other French Polynesian)
82080	Tongan
82990	Australasian nec
444444	Not classifiable
777777	Refusal*
888888	Don't know*
999999	No answer*
555555	No second ancestry* """

ethnicity_dict = {}
for line in ethnicities.split('\n'):
    if line:
        code, ethnicity = line.split('\t')
        ethnicity_dict[int(code)] = ethnicity

print(ethnicity_dict)

{10000: 'European', 11000: 'West European', 11010: 'Austrian nfs', 11018: 'Austrian city or region nec', 11019: 'Austrian nec', 11020: 'Belgian nfs', 11021: 'Flemish', 11022: 'Walloon', 11023: 'Brussels/Bruxellois', 11028: 'Belgian city or region nec', 11029: 'Belgian nec', 11030: 'British nfs', 11031: 'English', 11032: 'Northern Irish', 11033: 'Scottish', 11034: 'Ulster', 11035: 'Welsh', 11038: 'British city or region nec', 11039: 'British nec', 11040: 'Dutch nfs', 11048: 'Dutch city or region nec', 11049: 'Dutch nec', 11050: 'French nfs', 11051: 'Breton', 11052: 'Corse', 11058: 'French city or region nec', 11059: 'French nec', 11060: 'Frisian', 11070: 'German nfs', 11077: 'German diaspora/ethnic Germans', 11078: 'German city or region nec', 11079: 'German nec', 11080: 'Irish nfs', 11081: 'Irish traveller', 11088: 'Irish city or region nec', 11089: 'Irish nec', 11090: 'Luxembourgish nfs', 11098: 'Luxembourgish city or region nec', 11099: 'Luxembourgish nec', 11100: 'Swiss nfs', 11101:

In [9]:
df[['anctry1', 'anctry2']] = df[['anctry1', 'anctry2']].replace(ethnicity_dict)

In [10]:
df['imbgeco'] = (df['imbgeco'] - 0) / (10 - 0) * (4 - 1) + 1
df['imbgeco'] = 4 - df['imbgeco']

In [11]:
df = df.dropna()

In [12]:
df.to_csv('../data/clean-ess.csv', index=False)